## Quantization Error (QE) and Topographic Error (TE)

In [ ]:
# Quantization Error (QE)
# Average distance of a data point to the nearest lattice node
# Measures how well the mapping fits the distribution of the data

def som_quantization_error(data, trained_som):
    '''
    Calculates the quantization error of a trained SOMNet object

    Args:
        data: NumPy array of input data.
        trainded_som: trained SimpSOM SOMNet object

    Returns:
        quantization error (float)
    '''
    # Convert to clean NumPy array with uniform dtype first
    data_np = np.asarray(data, dtype=np.float64)

    # Convert NumPy array to CuPy array for GPU processing
    data_cp = cp.array(data_np)

    # Find all BMU indices
    bmu_indices = trained_som.find_bmu_ix(data_cp)

    # Get the weights of the BMUs
    bmu_weights = cp.array([trained_som.nodes_list[int(bmu_idx)].weights for bmu_idx in bmu_indices])

    # Vectorized distance calculation
    distances = cp.linalg.norm(bmu_weights - data_cp, axis=1)
    total_distance = cp.sum(distances)

    quantization_error = float(total_distance / len(data))
    return quantization_error

In [ ]:
# Topographic Error (TE)
# Proportion of data points whose BMU and second BMU are NOT neighbors
# Measures how well the shape of the data is preserved in the output space

def som_topographic_error(data, trained_som):
    """
    Computes topographic error ET for a SOM with hex topology using GPU (CuPy).
    
    Args:
        data:    The input data used to train the SOM (NumPy array).
        trained_som: A trained SOMNet object from the simpsom library.

    Returns:
        Topographic error (float between 0 and 1).
        Proportion of data points whose BMU and second BMU are NOT neighbors
    """
    # Convert data to CuPy arrays
    data_cp = cp.array(data)

    # Get all the nodes' weights from the trained SOM (GPU-enabled)
    weights_cp = cp.array([node.weights for node in trained_som.nodes_list])

    # Initialize total error
    total_error = 0

    # Find all BMU indices
    bmu_indices = trained_som.find_bmu_ix(data_cp)

    # Find all the second BMU indices
    som_dist = sps.distances.Distance(xp=np) # Initialize the distance object, as required by the documentation
    distances = som_dist.pairdist(data_cp, weights_cp, metric='euclidean')

    sorted_indices = cp.argsort(distances, axis=1)
    sbmu_indices = sorted_indices[:, 1]


    # Get the positions of the BMU and second BMU in the grid
    bmu_positions = cp.array([trained_som.nodes_list[int(bmu_idx)].pos for bmu_idx in bmu_indices])
    sbmu_positions = cp.array([trained_som.nodes_list[int(sbmu_idx)].pos for sbmu_idx in sbmu_indices])


    bmu_row, bmu_col = bmu_positions[:, 0], bmu_positions[:, 1]
    sbmu_row, sbmu_col = sbmu_positions[:, 0], sbmu_positions[:, 1]

    # Check if BMU and sBMU are neighbors
    row_neighbors = cp.abs(bmu_row - sbmu_row)
    col_neighbors = cp.abs(bmu_col - sbmu_col)

    not_neighbors = (row_neighbors > 1) | (col_neighbors > 1)

    number_not_neighbors = cp.sum(not_neighbors)

    # Compute the topographic error
    topographic_error = number_not_neighbors / data_cp.shape[0]

    return float(topographic_error)